# Path

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
from matplotlib import pyplot as plt
import pickle
import networkx as nx
import datetime
from datetime import timedelta
import seaborn as sns
from itertools import combinations
sns.set_theme(style="ticks")

In [2]:
df = pd.read_csv('dataset/final_dataset.csv', parse_dates = ['Timestamp'])
df['Total_Price'] = round(df['Price'] * df['Swap_Value'], 2)
df

,_ID,Timestamp,From,From_Chain_ID_1,To_Chain_ID_1,Swap_Token,Value,Fee,Swap_Value,Time_Execution,Confirmations,Swap_Token_Symbol,Price,Total_Price
0,6213551711d45e0a4968cfa8,2022-02-21 09:03:50,0x0afd7eddde20542783a64302ce47426fbe497feb,BNB CHAIN,Fantom,anyUSDC,1920.060000,0.90000,1919.160000,44.64,27,USDC,1.004421,1927.64
1,621355609e1cf703a57a9695,2022-02-21 09:03:50,0xbc26b49572512da9e81f358c1218780a57922ed8,Avalanche,Fantom,MIM,45.076839,0.90000,44.176839,37.42,21,MIM,0.994921,43.95
2,621355209e1cf703a57a968b,2022-02-21 09:03:46,0x92630d4c22b8d2232ac11c0ad713051654ee9e3a,Avalanche,BNB CHAIN,bnbv5,0.249163,0.01400,0.235163,89.64,15,BNB,381.575350,89.73
3,621355219e1cf703a57a968d,2022-02-21 09:03:00,0x272f6806da025b58bebbbbb1edea01bd6cff45e7,Fantom,Avalanche,anyAVAX,0.522815,0.08000,0.442815,76.26,52,AVAX,78.122940,34.59
4,621355229e1cf703a57a9692,2022-02-21 09:02:52,0x5628a6b42db55cdf8d051780e568a232ee6d4cdc,BNB CHAIN,Fantom,anyUSDT,79.000000,0.90000,78.100000,54.91,65,USDT,1.001046,78.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72607,620918d5d3ea681f35eb8850,2022-02-13 14:43:11,0xeed31690621b37d37f0b8e32dfb822674f2a3f42,Avalanche,BNB CHAIN,bnbv5,2.020000,0.01400,2.006000,77.96,17,BNB,403.395780,809.21
72608,620918d7d3ea681f35eb8855,2022-02-13 14:43:07,0x26dbd93c5261ec44ecbfea71d0992b95b6ef147e,Fantom,BNB CHAIN,bnb,4.789489,0.02200,4.767489,87.66,22,BNB,403.395780,1923.18
72609,620917cd2219a24b7e94ae72,2022-02-13 14:43:05,0x0d4b70dd05e02a18335ae3308b51011d491f758b,Fantom,Polygon,anyTOMB,3039.510000,3.03951,3036.470490,292.62,143,TOMB,1.958229,5946.10
72610,620918d5d3ea681f35eb8853,2022-02-13 14:42:45,0xf7cd7674485f559779b7d19860bce3710fce378f,Avalanche,Fantom,anyAVAX,64.000000,0.08000,63.920000,27.36,72,AVAX,81.397550,5202.93


In [3]:
with open('data_complete.p', 'rb') as fp:
    trans_dict = pickle.load(fp)

In [4]:
# the function returns the longest path_contiguous with all the arcs in it

def get_path_contiguo(archi,l):
    # start_edge is the arc with id 1
    start_edge = archi[0]
    # archi visitati
    visited = set()
    # arches visited
    to_visit = [] #queue
    to_visit.append(start_edge)

    # contiguous path that I am currently building
    path_contiguo = [start_edge, ]
    paths=[]
    while to_visit:
        
        found_next_edge = False
        current_edge = to_visit.pop()
                
        try:
            next_edge = archi[archi.index(current_edge)+1]
            
        except Exception as e:
            #if I finish here I have run out of arcs to analyze and I have to go out the loop
            break
        if current_edge[1] == next_edge[0]:
            found_next_edge = True
            to_visit.append(next_edge)
            path_contiguo.append(next_edge)
        
        if not found_next_edge:
    
            to_visit.append(next_edge)
            paths.append(path_contiguo)
            
            path_contiguo= []
            path_contiguo.append(next_edge)
    
    try:
        le = max(len(x) for x in paths)   #find out the max length
        #longer_path=max(paths, key=len)
        longer_paths = [x for x in paths if len(x) == le]  #now filter list based on that max length
    except Exception as e:
        return []
    
    for p in longer_paths:
        
        longerp = [p[i: j] for i in range(len(p)) for j in range(i + 1, len(p) + 1) if len(p[i:j]) == l]
          
    return longerp

In [5]:
def checkpath(path,hour,l):
    if len(path) < l:
        return False
    i=0
    
    for i in range(len(path)-1):
        ts_1=df[df['_ID']==path[i][2]]['Timestamp'].iloc[0]    # I get the ts of the transaction from '_ID'
      
        ts_2=df[df['_ID']==path[i+1][2]]['Timestamp'].iloc[0]  # I get the ts of the transaction from '_ID'
        
        #if ( ts_1 > ts_2): # path transactions must be temporally ordered (no need because the dataset is ordered)
        #    return False
        # between transactions there must be at most timedelta time
        if ( ts_2 - ts_1 ) > timedelta(hours=hour):
            return False
        i=i+1

    return True

In [ ]:
ore=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,2,6,12,18,24]


for l in [4,5,6,7]:
    print('l:',l)
    for h in ore: 

        addresses = []
        token = []
        BC_1 = []
        BC_2 = []
        tx_id_1 = []
        tx_id_2=[]
        BC_3 = []
        tx_id_3 = []
        BC_4=[]
        tx_id_4 = []
        BC_5=[]
        tx_id_5 = []
        BC_6=[]
        tx_id_6 = []
        BC_7=[]
        tx_id_7 = []
        BC_8=[]
        #count=0

        for  a in trans_dict:   

            for t in trans_dict[a]:  # transactions made from the same address

                G = [] 
                if len(trans_dict[a][t]) >1:    #if only one transaction was made with that token there can be no path
                    for i in trans_dict[a][t]:   #transactions of the same address with the same token
                        G.append([i[3], i[4], i[0]])

                    long_path=get_path_contiguo(G,l)
                    for path in long_path : 
                        if checkpath(path,h,l): #checks the length of the pathcontiguous and that transactions are temporally ordered
                            addresses.append(a)
                            token.append(t)
                            BC_1.append(path[0][0])
                            tx_id_1.append(path[0][2])
                            BC_2.append(path[1][0])
                            tx_id_2.append(path[1][2])
                            BC_3.append(path[1][1])
                            #path da 3
                            if l>=3:
                                tx_id_3.append(path[2][2])
                                BC_4.append(path[2][1])
                                #path da 4
                                if l>=4:
                                    tx_id_4.append(path[3][2])
                                    BC_5.append(path[3][1])
                                    #path da 5
                                    if l>=5:
                                        tx_id_5.append(path[4][2])
                                        BC_6.append(path[4][1])
                                        #path da 6
                                        if l>=6:
                                            tx_id_6.append(path[5][2])
                                            BC_7.append(path[5][1])
                                            #path da 7
                                            if l==7:
                                                tx_id_7.append(path[6][2])
                                                BC_8.append(path[6][1])

        if l==2:
            #path da 2 swaps
            df_path = pd.DataFrame(list(zip( addresses, token, BC_1, BC_2, BC_3, tx_id_1, tx_id_2)), columns = [ 'Addresses', 'Token', 'Blockchain_1', 'Blockchain_2', 'Blockchain_3', 'TxID_1', 'TxID_2']) 
        elif l==3:
            #path da 3 swaps
            df_path = pd.DataFrame(list(zip( addresses, token, BC_1, BC_2, BC_3, BC_4, tx_id_1, tx_id_2, tx_id_3)), columns = [ 'Addresses', 'Token', 'Blockchain_1', 'Blockchain_2', 'Blockchain_3', 'Blockchain_4', 'TxID_1', 'TxID_2', 'TxID_3']) 
        elif l==4:
            #path da 4
            df_path = pd.DataFrame(list(zip( addresses, token, BC_1, BC_2, BC_3, BC_4, BC_5, tx_id_1, tx_id_2, tx_id_3, tx_id_4)), columns = [ 'Addresses', 'Token', 'Blockchain_1', 'Blockchain_2', 'Blockchain_3', 'Blockchain_4', 'Blockchain_5', 'TxID_1', 'TxID_2', 'TxID_3', 'TxID_4']) 
        elif l==5:
            #path da 5
            df_path = pd.DataFrame(list(zip( addresses, token, BC_1, BC_2, BC_3, BC_4, BC_5, BC_6, tx_id_1, tx_id_2, tx_id_3, tx_id_4, tx_id_5)), columns = [ 'Addresses', 'Token', 'Blockchain_1', 'Blockchain_2', 'Blockchain_3', 'Blockchain_4', 'Blockchain_5', 'Blockchain_6', 'TxID_1', 'TxID_2', 'TxID_3', 'TxID_4', 'TxID_5']) 
        elif l==6:
            #path da 6
            df_path = pd.DataFrame(list(zip( addresses, token, BC_1, BC_2, BC_3, BC_4, BC_5, BC_6, BC_7, tx_id_1, tx_id_2, tx_id_3, tx_id_4, tx_id_5, tx_id_6)), columns = [ 'Addresses', 'Token', 'Blockchain_1', 'Blockchain_2', 'Blockchain_3', 'Blockchain_4', 'Blockchain_5', 'Blockchain_6', 'Blockchain_7', 'TxID_1', 'TxID_2', 'TxID_3', 'TxID_4', 'TxID_5', 'TxID_6'])
        elif l==7:
            #path da 7
            df_path = pd.DataFrame(list(zip( addresses, token, BC_1, BC_2, BC_3, BC_4, BC_5, BC_6, BC_7, BC_8, tx_id_1, tx_id_2, tx_id_3, tx_id_4, tx_id_5, tx_id_6, tx_id_7)), columns = [ 'Addresses', 'Token', 'Blockchain_1', 'Blockchain_2', 'Blockchain_3', 'Blockchain_4', 'Blockchain_5', 'Blockchain_6', 'Blockchain_7',  'Blockchain_8', 'TxID_1', 'TxID_2', 'TxID_3', 'TxID_4', 'TxID_5', 'TxID_6', 'TxID_7'])

        df_path.to_csv(f'longest_pathsv2/path_L{l}_H{h}.csv')
